# Imports

In [ ]:
from matplotlib.patches  import Rectangle
from matplotlib          import ticker
from IPython.display     import Image
from scipy.interpolate   import interp2d
from scipy.integrate     import odeint
from matplotlib          import rcParams
from brewer2mpl          import get_map
from aux                 import *

%pylab

rcParams['axes.linewidth']  = 2
rcParams['lines.linewidth'] = 2

# Set parameters

In [ ]:
M_star = 0.8*M_sun
M_disk = 0.01*M_star
r_d    = 200*AU
r_0    = 0.1*AU
p      = 1 # needed for analytical solution
alpha  = 1e-3
rho_s  = 1.2
v_frag = 1000.
n_m    = 100
n_r    = 200
q      = 0.5
TAU    = 200.
#
# define the dust-to-gas ratio
#
d2g0   = 5e-3 # the dust to gas ratio of the outer disk, using eps ~r**0.25
#eps_f  = lambda r: min(d2g0*(r/r_c)**0.25,d2g0)
eps_f  = lambda r: d2g0

r      = logspace(log10(r_0/AU),log10(r_d/AU),n_r)*AU
a      = logspace(-4,3,n_m)
m      = 4*pi/3.*rho_s*a**3 
#
# calculate normalization for sigma_gas
#
sig_norm = (2-p)*M_disk/(2*pi*r_d**2)*(1-(r_0/r_d)**(2-p))**-1

## Calculate trajectories $y(t)$, where $y = [a, r]$

Define lambda functions.

In [ ]:
p_g_f   = lambda r: p                                             # gas surface density index
p_v_gas = lambda r: -p_g_f(r)-q+2.                                # index used in the gas velocity
gamma_f = lambda r: -p_g_f(r)-(q+3.)/2.                           # pressure index
sig_g_f = lambda r: sig_norm*(r/r_d)**-p                          # gas surface density
T_f     = lambda r: 200*(r/AU)**-0.5                              # temperature
cs_f    = lambda r: sqrt(k_b*T_f(r)/mu/m_p)                       # sound speed
om_f    = lambda r: sqrt(Grav*M_star/r**3)                        # kepler velocity
v_g_f   = lambda r: -3*alpha*cs_f(r)**2/om_f(r)/r*p_v_gas(r)      # gas velocity

In [ ]:
St_f    = lambda r,a: a*rho_s/sig_g_f(r)*pi/2.                                         # Stokes number
v_gd_f  = lambda r,a: 1./(1.+St_f(r,a)**2)*v_g_f(r)                                    # gas drag velocity
v_dr_f  = lambda r,a: 1./(St_f(r,a)+St_f(r,a)**-1)*(cs_f(r)**2/(r*om_f(r))*gamma_f(r)) # drift velocity
v_t_f   = lambda r,a: v_dr_f(r,a) + v_gd_f(r,a)                                        # total dust velocity
         
a_fr_f  = lambda r:  2./(3.*pi)*sig_g_f(r)/(rho_s*alpha)*v_frag**2/cs_f(r)**2          #  simple frag. barrier
a_st_f  = lambda r: 2*sig_g_f(r)/(pi*rho_s)                                            # St=1 line
a_dr_f  = lambda r: 0.55*2*sig_g_f(r)*eps_f(r)*r**2*om_f(r)**2/ \
                    (pi*rho_s*cs_f(r)**2*abs(gamma_f(r)))                              # drift limit

b       = lambda r: 3.*alpha*(cs_f(r)/v_frag)**2                                       # quadratic fragmentation barrier
a_fr_f2 = lambda r: sig_g_f(r)/(pi*rho_s)*(b(r)-sqrt(b(r)**2-4.))

#
# drift velocity
#
drdt    = lambda r,a: v_t_f(r,a)
#
# growth rate, using hd = hg*min(1,sqrt(alpha/St))
#
dadt = lambda r,a: eps_f(r)*sig_g_f(r)*om_f(r)/rho_s*sqrt(3./(2.*pi)) * sqrt(alpha/(St_f(r,a)+St_f(r,a)**-1))/min(1.,sqrt(alpha/St_f(r,a)))

Define Analytical Solution

In [ ]:
def sol(r0,a0,t=None,F=1e-2,N=200,t0=1.):
    """
    Calculate the drift+growth trajectory [r(t),a(t)]. If `t` is given, it will calculate
    the solution on that time array, otherwise it will integrate until a time `t1` when the
    trajectory reaches a radius r(t1)=F*r0.
    
    Arguments:
    ----------
    
    r0 : float
    :   initial radius [AU]
    
    a0 : float
    :   initial grain size [cm]
    
    Keywords:
    ---------
    
    t : None|array
    :   the time array on which the solution is calculated, if None: see other keywords
    
    F : float
    :   the fraction of the initial radius until which the integration is carried out
    
    N : int
    :   the number of time snap shots of the solution array
    
    t0 : float
    :   the first time snap shot of the logarithmic time array
    
    Returns:
    --------
    
    sol, t
    
    sol : array
    :   sol[:,0] are the radii r(t)             [cm]
        sol[:,1] are the grain sizes a(t)       [cm]
        t[:]     are the times of the snapshots [s]
    """
    A = a0*rho_s*pi/2./sig_g_f(r0)*(cs_f(r0)/om_f(r0)/r0)**2*om_f(r0)*gamma_f(r0)
    B = eps_f(r0)*om_f(r0)
    
    if t is None:
        t1=(2.*log(-((3.*A + 2.*B)*((2.*B)/(3.*A + 2.*B) - F**1.5))/(3.*A)))/(3.*A + 2.*B)
        t=append(0,np.logspace(np.log10(t0),np.log10(t1),N))
        

    x = lambda t: (   (2.*B+3.*A*exp(1.5*A*t+B*t)) / (3.*A+2.*B)  )**(2./3.)
    y = lambda t: ((3.*A + 2.*B)*exp((3.*A*t)/2. + B*t))/(2.*B + 3.*A*exp((3.*A*t)/2. + B*t))
    return np.array([a0*y(t),r0*x(t)]).T,t

## Integration: analytical solution

In [ ]:
SOL  = []
SOLT = []

for r0 in [1,10,100]:
    print('r0 = {} AU'.format(r0))
    r0   = r0*AU
    a0   = 1e-4
    y0   = array([a0,r0])
    #
    #  define the derivative dy/dt and integrate
    #
    dydt  = lambda y,t: [dadt(y[1],y[0]),drdt(y[1],y[0])]
    s,tt  = sol(r0,a0,N=300,F=0.5e-2)
    SOL  += [s]
    SOLT += [tt]

## Plot the velocity arrows and trajectories

In [ ]:
cols = get_map('Dark2','Qualitative',3).mpl_colors

dadt_v  = np.vectorize(dadt)
v_t_f_v = np.vectorize(v_t_f)

i_q = 10
X,Y = meshgrid(r,a)
U   = r[newaxis,:]/v_t_f_v(*meshgrid(r,a))/year                 # drift time scale in years
V   = a[:,newaxis]/dadt_v(*meshgrid(r,a))/year # growth time scale in years
#
# limit to between 1e3 and 1e6 years
#
U   = sign(U)*maximum(minimum(abs(U),1e6),1e2)
V   = sign(V)*maximum(minimum(abs(V),1e6),1e2)
# scale such that 0 = 1e6
#                 1 = 1e5
#                 2 = 1e4
#                 3 = 1e3
#                 4 = 1e2
Vs = (6-log10(abs(V)))*sign(V)
Us = (6-log10(abs(U)))*sign(U)
#
# set quiver properties
#
props={'scale':1/0.01,'units':'width','width':0.002}
#
# draw the arrows
#
figure()
Qlr = quiver(X[::i_q,::i_q]/AU,Y[::i_q,::i_q],Us[::i_q,::i_q],zeros(Vs[::i_q,::i_q].shape),color='0.5',**props)
Qud = quiver(X[::i_q,::i_q]/AU,Y[::i_q,::i_q],zeros(Us[::i_q,::i_q].shape),Vs[::i_q,::i_q],color='0.5',**props)
Qb  = quiver(X[::i_q,::i_q]/AU,Y[::i_q,::i_q],Us[::i_q,::i_q],Vs[::i_q,::i_q],color='r',alpha=0.5,**props)
xscale('log')
yscale('log')
xlabel('radius [AU]')
ylabel('particle size [cm]')
xlim(0.2,500);
ylim(a[0],a[-1]);
#
# draw lines
#
loglog(r/AU,a_st_f(r),'k',zorder=0,label='St=1')
loglog(r/AU,a_fr_f2(r),'-',  zorder=0,c=cols[0],label='$a_\mathrm{frag}$')
loglog(r/AU,a_dr_f(r),'-',  zorder=0,c=cols[1],label='$a_\mathrm{drift}$')
#
# draw trajectories
#
for i,s in enumerate(zip(SOL,SOLT)):
    tt=s[1]
    s=s[0]
    loglog(s[:,1]/AU,s[:,0],'--',c=cols[2],label=(i==1)*'trajectory')
#
# draw box for scale arrows
#
xo,yo=0.71,0.35
rect = Rectangle((xo,yo+0.03), 0.27, 0.6, facecolor="0.7",alpha=0.9,transform=gca().transAxes,zorder=3)
gca().minorticks_off()
gca().add_patch(rect)
#
# draw scale arrows
#
kprops={'labelpos':'E','color':'r','alpha':1}
qk1 = quiverkey(Qb,xo+0.05,yo+0.065*5,4,r'$10^2$ years',**kprops)
qk2 = quiverkey(Qb,xo+0.05,yo+0.065*4,3,r'$10^3$ years',**kprops)
qk3 = quiverkey(Qb,xo+0.05,yo+0.065*3,2,r'$10^4$ years',**kprops)
qk4 = quiverkey(Qb,xo+0.05,yo+0.065*2,1,r'$10^5$ years',**kprops)
qk5 = quiverkey(Qb,xo+0.05,yo+0.065*1,0,r'$10^6$ years',**kprops)
for qk in [qk1,qk2,qk3,qk4,qk5]: qk.set_zorder(4)
legend(loc=(0.7,0.73),fontsize=14).get_frame().set_alpha(0)
#
# save figure
#
gca().set_axis_bgcolor('none')
gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
savefig('velocity_arrows.pdf',facecolor='none')
savefig('velocity_arrows.png',facecolor='none')
#close()
display(Image(filename='velocity_arrows.png'))